In [1]:
import pandas as pd;
import numpy as np;
import csv

In [2]:
# read in dataset

transaction=pd.read_csv("/Users/wenxinzhang/Desktop/graduate/course/2nd_semester/AML/proj/h-and-m-personalized-fashion-recommendations/transactions_train.csv")
transaction.head()


In [3]:
transaction.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [372]:
len(set(transaction['customer_id']))

1362281

In [347]:
df=transaction
df['t_dat']=pd.to_datetime(df['t_dat'])
df['year']=df['t_dat'].dt.year
df['month']=df['t_dat'].dt.month
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id,year,month
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,2018,9
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,2018,9
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,2018,9
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,2018,9
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2,2018,9


## generate subset of transactions according to the transaction time

In [348]:
df_18_09=df.loc[(df.year== 2018) & (df.month == 9)]
df_18_10=df.loc[(df.year== 2018) & (df.month == 10)]
df_18_11=df.loc[(df.year== 2018) & (df.month == 11)]
df_18_12=df.loc[(df.year== 2018) & (df.month == 12)]
df_19_01=df.loc[(df.year== 2019) & (df.month == 1)]
df_19_02=df.loc[(df.year== 2019) & (df.month == 2)]
df_19_03=df.loc[(df.year== 2019) & (df.month == 3)]
df_19_04=df.loc[(df.year== 2019) & (df.month == 4)]
df_19_05=df.loc[(df.year== 2019) & (df.month == 5)]
df_19_06=df.loc[(df.year== 2019) & (df.month == 6)]
df_19_07=df.loc[(df.year== 2019) & (df.month == 7)]
df_19_08=df.loc[(df.year== 2019) & (df.month == 8)]
df_19_09=df.loc[(df.year== 2019) & (df.month == 9)]
df_19_10=df.loc[(df.year== 2019) & (df.month == 10)]
df_19_11=df.loc[(df.year== 2019) & (df.month == 11)]
df_19_12=df.loc[(df.year== 2019) & (df.month == 12)]
df_20_01=df.loc[(df.year== 2020) & (df.month == 1)]
df_20_02=df.loc[(df.year== 2020) & (df.month == 2)]
df_20_03=df.loc[(df.year== 2020) & (df.month == 3)]
df_20_04=df.loc[(df.year== 2020) & (df.month == 4)]
df_20_05=df.loc[(df.year== 2020) & (df.month == 5)]
df_20_06=df.loc[(df.year== 2020) & (df.month == 6)]
df_20_07=df.loc[(df.year== 2020) & (df.month == 7)]
df_20_08=df.loc[(df.year== 2020) & (df.month == 8)]
df_20_09=df.loc[(df.year== 2020) & (df.month == 9)]

## Define Functions

In [349]:
# get unique customer_id for each year and sort them
def get_customer_id(id_list):
    df= pd.DataFrame(set(id_list),columns=['customer_id'])
    df=df.sort_values(by='customer_id' , ascending=False) 
    df=df.reset_index(drop=True)
    return df

# sort the transaction info according to customer_id
def sort_transaction(df):
    df=df.sort_values(by='customer_id' , ascending=False) 
    df=df.reset_index(drop=True)
    return df

# get the purchasing list for each customer
def purchasing_list(df_id, df_trans):
    # both of df_id and df_trans are ordered by the customer ID
    total_purchasing_list=[]
    
    for i in range(len(df_id)):
        current_id=df_id['customer_id'][i]
        current_list=[]
        for j in range(len(df_trans)):
            if current_id==df_trans['customer_id'][j]:
                current_list.append(df_trans['article_id'][j])
                drop_rows=j+1
            else: break 
        df_trans.drop(df_trans.head(drop_rows).index,inplace=True) 
        df_trans=df_trans.reset_index(drop=True)
        total_purchasing_list.append(current_list)
    return total_purchasing_list

# get the unique purchasing list for each customer
def remove_duplicate(purchasing_list):
    unique_list=purchasing_list.copy()
    for i in range(len(unique_list)):
        unique_list[i]=list(set(unique_list[i]))
    return unique_list

# get the most frequent purchasing item for each customer
def most_frequent_per_id(item_list):
    max_ocur = max([item_list.count(i) for i in item_list])
    if max_ocur>=2:
        return [i for i in set(item_list) if item_list.count(i) == max_ocur]
    # NA indicates no special preference for the purchasing items

# Combine most frequent purchasing item for all customer
def most_frequent(purchasing_list):
    most_frequent_list=[]
    for i in range(len(purchasing_list)):
        item=most_frequent_per_id(purchasing_list[i])
        most_frequent_list.append(item)
    return most_frequent_list

# write seperate file to csv
def write_to_csv(df,purch_list,file_name):
    list_column=["customer_id","purchasing_list"]
    column_A= df["customer_id"]
    column_B= purch_list
    list_row=[]

    #to create a list full of rows as the writerow function reads data row-wise
    for i in range(len(column_A)):
        list_temp=[column_A[i],column_B[i]]
        list_row.append(list_temp)


    with open (file_name, 'w', newline="") as entry:
        writer=csv.writer(entry)
        writer.writerow(list_column)
        writer.writerows(list_row)

        entry.close()
        
# Merge files  (start from second merge)
def merge_dataset(df1,df2):
    df=pd.merge(df1,df2,on='customer_id',how='outer')
    df['purchasing_list']=""
    df=df.fillna("")
    for i in range(len(df)):
        if len(df['purchasing_list_x'][i])==0:
            df['purchasing_list'][i]=df['purchasing_list_y'][i][1:-1]
        elif len(df['purchasing_list_y'][i])==0:
            df['purchasing_list'][i]=df['purchasing_list_x'][i]
        else:
            df['purchasing_list'][i]=df['purchasing_list_x'][i]+', '+df['purchasing_list_y'][i][1:-1]
    df=df.drop(['purchasing_list_x','purchasing_list_y'], axis=1)
    return df






## generate seperate datasets and write to csv

In [143]:
df_18_09_id=get_customer_id(df_18_09.customer_id)
df_18_09_trans=sort_transaction(df_18_09)
purchasing_18_09=purchasing_list(df_18_09_id, df_18_09_trans)
write_to_csv(df_18_09_id,purchasing_18_09,'2018-09.csv')

In [ ]:
df_18_10_id=get_customer_id(df_18_10.customer_id)
df_18_10_trans=sort_transaction(df_18_10)
purchasing_18_10=purchasing_list(df_18_10_id, df_18_10_trans)
write_to_csv(df_18_10_id,purchasing_18_10,'2018-10.csv')

In [ ]:
df_18_11_id=get_customer_id(df_18_11.customer_id)
df_18_11_trans=sort_transaction(df_18_11)
purchasing_18_11=purchasing_list(df_18_11_id, df_18_11_trans)
write_to_csv(df_18_11_id,purchasing_18_11,'2018-11.csv')

In [ ]:
df_18_12_id=get_customer_id(df_18_12.customer_id)
df_18_12_trans=sort_transaction(df_18_12)
purchasing_18_12=purchasing_list(df_18_12_id, df_18_12_trans)
write_to_csv(df_18_12_id,purchasing_18_12,'2018-12.csv')

In [ ]:
df_19_01_id=get_customer_id(df_19_01.customer_id)
df_19_01_trans=sort_transaction(df_19_01)
purchasing_19_01=purchasing_list(df_19_01_id, df_19_01_trans)
write_to_csv(df_19_01_id,purchasing_19_01,'2019-01.csv')

In [ ]:
df_19_02_id=get_customer_id(df_19_02.customer_id)
df_19_02_trans=sort_transaction(df_19_02)
purchasing_19_02=purchasing_list(df_19_02_id, df_19_02_trans)
write_to_csv(df_19_02_id,purchasing_19_02,'2019-02.csv')

In [ ]:
df_19_03_id=get_customer_id(df_19_03.customer_id)
df_19_03_trans=sort_transaction(df_19_03)
purchasing_19_03=purchasing_list(df_19_03_id, df_19_03_trans)
write_to_csv(df_19_03_id,purchasing_19_03,'2019-03.csv')

In [ ]:
df_19_04_id=get_customer_id(df_19_04.customer_id)
df_19_04_trans=sort_transaction(df_19_04)
purchasing_19_04=purchasing_list(df_19_04_id, df_19_04_trans)
write_to_csv(df_19_04_id,purchasing_19_04,'2019-04.csv')

In [350]:
df_19_05_id=get_customer_id(df_19_05.customer_id)
df_19_05_trans=sort_transaction(df_19_05)
purchasing_19_05=purchasing_list(df_19_05_id, df_19_05_trans)
write_to_csv(df_19_05_id,purchasing_19_05,'2019-05.csv')

In [351]:
df_19_06_id=get_customer_id(df_19_06.customer_id)
df_19_06_trans=sort_transaction(df_19_06)
purchasing_19_06=purchasing_list(df_19_06_id, df_19_06_trans)
write_to_csv(df_19_06_id,purchasing_19_06,'2019-06.csv')

In [352]:
df_19_07_id=get_customer_id(df_19_07.customer_id)
df_19_07_trans=sort_transaction(df_19_07)
purchasing_19_07=purchasing_list(df_19_07_id, df_19_07_trans)
write_to_csv(df_19_07_id,purchasing_19_07,'2019-07.csv')

In [353]:
write_to_csv(df_19_05_id,purchasing_19_05,'2019-05.csv')
write_to_csv(df_19_06_id,purchasing_19_06,'2019-06.csv')
write_to_csv(df_19_07_id,purchasing_19_07,'2019-07.csv')

In [354]:
df_19_08_id=get_customer_id(df_19_08.customer_id)
df_19_08_trans=sort_transaction(df_19_08)
purchasing_19_08=purchasing_list(df_19_08_id, df_19_08_trans)
write_to_csv(df_19_08_id,purchasing_19_08,'2019-08.csv')

In [355]:
df_19_09_id=get_customer_id(df_19_09.customer_id)
df_19_09_trans=sort_transaction(df_19_09)
purchasing_19_09=purchasing_list(df_19_09_id, df_19_09_trans)
write_to_csv(df_19_09_id,purchasing_19_09,'2019-09.csv')

In [356]:
df_19_10_id=get_customer_id(df_19_10.customer_id)90
df_19_10_trans=sort_transaction(df_19_10)
purchasing_19_10=purchasing_list(df_19_10_id, df_19_10_trans)
write_to_csv(df_19_10_id,purchasing_19_10,'2019-10.csv')

In [357]:
df_19_11_id=get_customer_id(df_19_11.customer_id)
df_19_11_trans=sort_transaction(df_19_11)
purchasing_19_11=purchasing_list(df_19_11_id, df_19_11_trans)
write_to_csv(df_19_11_id,purchasing_19_11,'2019-11.csv')

In [358]:
df_19_12_id=get_customer_id(df_19_12.customer_id)
df_19_12_trans=sort_transaction(df_19_12)
purchasing_19_12=purchasing_list(df_19_12_id, df_19_12_trans)
write_to_csv(df_19_12_id,purchasing_19_12,'2019-12.csv')

In [360]:
df_20_01_id=get_customer_id(df_20_01.customer_id)
df_20_01_trans=sort_transaction(df_20_01)
purchasing_20_01=purchasing_list(df_20_01_id, df_20_01_trans)
write_to_csv(df_20_01_id,purchasing_20_01,'2020-01.csv')

In [361]:
df_20_02_id=get_customer_id(df_20_02.customer_id)
df_20_02_trans=sort_transaction(df_20_02)
purchasing_20_02=purchasing_list(df_20_02_id, df_20_02_trans)
write_to_csv(df_20_02_id,purchasing_20_02,'2020-02.csv')

In [362]:
df_20_03_id=get_customer_id(df_20_03.customer_id)
df_20_03_trans=sort_transaction(df_20_03)
purchasing_20_03=purchasing_list(df_20_03_id, df_20_03_trans)
write_to_csv(df_20_03_id,purchasing_20_03,'2020-03.csv')

In [363]:
df_20_04_id=get_customer_id(df_20_04.customer_id)
df_20_04_trans=sort_transaction(df_20_04)
purchasing_20_04=purchasing_list(df_20_04_id, df_20_04_trans)
write_to_csv(df_20_04_id,purchasing_20_04,'2020-04.csv')

In [364]:
df_20_05_id=get_customer_id(df_20_05.customer_id)
df_20_05_trans=sort_transaction(df_20_05)
purchasing_20_05=purchasing_list(df_20_05_id, df_20_05_trans)
write_to_csv(df_20_05_id,purchasing_20_05,'2020-05.csv')

In [365]:
df_20_06_id=get_customer_id(df_20_06.customer_id)
df_20_06_trans=sort_transaction(df_20_06)
purchasing_20_06=purchasing_list(df_20_06_id, df_20_06_trans)
write_to_csv(df_20_06_id,purchasing_20_06,'2020-06.csv')

In [366]:
df_20_07_id=get_customer_id(df_20_07.customer_id)
df_20_07_trans=sort_transaction(df_20_07)
purchasing_20_07=purchasing_list(df_20_07_id, df_20_07_trans)
write_to_csv(df_20_07_id,purchasing_20_07,'2020-07.csv')

In [367]:
df_20_08_id=get_customer_id(df_20_08.customer_id)
df_20_08_trans=sort_transaction(df_20_08)
purchasing_20_08=purchasing_list(df_20_08_id, df_20_08_trans)
write_to_csv(df_20_08_id,purchasing_20_08,'2020-08.csv')

In [368]:
df_20_09_id=get_customer_id(df_20_09.customer_id)
df_20_09_trans=sort_transaction(df_20_09)
purchasing_20_09=purchasing_list(df_20_09_id, df_20_09_trans)
write_to_csv(df_20_09_id,purchasing_20_09,'2020-09.csv')

## read in seperate datasets

In [369]:
df_2018_09=pd.read_csv("2018-09.csv")
df_2018_10=pd.read_csv("2018-10.csv")
df_2018_11=pd.read_csv("2018-11.csv")
df_2018_12=pd.read_csv("2018-12.csv")
df_2019_01=pd.read_csv("2019-01.csv")
df_2019_02=pd.read_csv("2019-02.csv")
df_2019_03=pd.read_csv("2019-03.csv")
df_2019_04=pd.read_csv("2019-04.csv")
df_2019_05=pd.read_csv("2019-05.csv")
df_2019_06=pd.read_csv("2019-06.csv")
df_2019_07=pd.read_csv("2019-07.csv")
df_2019_08=pd.read_csv("2019-08.csv")
df_2019_09=pd.read_csv("2019-09.csv")
df_2019_10=pd.read_csv("2019-10.csv")
df_2019_11=pd.read_csv("2019-11.csv")
df_2019_12=pd.read_csv("2019-12.csv")
df_2020_01=pd.read_csv("2020-01.csv")
df_2020_02=pd.read_csv("2020-02.csv")
df_2020_03=pd.read_csv("2020-03.csv")
df_2020_04=pd.read_csv("2020-04.csv")
df_2020_05=pd.read_csv("2020-05.csv")
df_2020_06=pd.read_csv("2020-06.csv")
df_2020_07=pd.read_csv("2020-07.csv")
df_2020_08=pd.read_csv("2020-08.csv")
df_2020_09=pd.read_csv("2020-09.csv")

## Merge the datasets

##### First Merge

In [263]:
df_merge1=pd.merge(df_2018_09,df_2018_10,on='customer_id',how='outer')

In [264]:
df_merge1['purchasing_list']=""
df_merge1.head()

,customer_id,purchasing_list_x,purchasing_list_y,purchasing_list
0,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,[568597019],"[484398001, 484398001]",
1,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,[671695001],NaN,
2,fffed13f041cc91f74dc6ec4e3267d7bb71bc833d53b7d...,[661435002],NaN,
3,fffec3dbcc87c78300f37f62cfca4274f1ea6ef59dba10...,"[559645001, 657097001]",[694153001],
4,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,"[580770002, 496111013, 548467001]","[523642005, 618480001, 548657001, 573856024, 6...",


In [265]:
df_merge1=df_merge1.fillna("")

In [266]:
for i in range(len(df_merge1)):
    if len(df_merge1['purchasing_list_x'][i])==0:
        df_merge1['purchasing_list'][i]=df_merge1['purchasing_list_y'][i][1:-1]
    elif len(df_merge1['purchasing_list_y'][i])==0:
        df_merge1['purchasing_list'][i]=df_merge1['purchasing_list_x'][i][1:-1]
    else:
        df_merge1['purchasing_list'][i]=df_merge1['purchasing_list_x'][i][1:-1]+', '+df_merge1['purchasing_list_y'][i][1:-1]


In [272]:
df_merge1=df_merge1.drop(['purchasing_list_x','purchasing_list_y'], axis=1)


##### Rest merge

In [ ]:
df_merge2=merge_dataset(df_merge1,df_2018_11)
df_merge3=merge_dataset(df_merge2,df_2018_12)
df_merge4=merge_dataset(df_merge3,df_2019_01)
df_merge5=merge_dataset(df_merge4,df_2019_02)
df_merge6=merge_dataset(df_merge5,df_2019_03)
df_merge7=merge_dataset(df_merge6,df_2019_04)
df_merge8=merge_dataset(df_merge7,df_2019_05)
df_merge9=merge_dataset(df_merge8,df_2019_06)
df_merge10=merge_dataset(df_merge9,df_2019_07)
df_merge11=merge_dataset(df_merge10,df_2019_08)
df_merge12=merge_dataset(df_merge11,df_2019_09)
df_merge13=merge_dataset(df_merge12,df_2019_10)
df_merge14=merge_dataset(df_merge13,df_2019_11)
df_merge15=merge_dataset(df_merge14,df_2019_12)
df_merge16=merge_dataset(df_merge15,df_2020_01)
df_merge17=merge_dataset(df_merge16,df_2020_02)
df_merge18=merge_dataset(df_merge17,df_2020_03)
df_merge19=merge_dataset(df_merge18,df_2020_04)
df_merge20=merge_dataset(df_merge19,df_2020_05)
df_merge21=merge_dataset(df_merge20,df_2020_06)
df_merge22=merge_dataset(df_merge21,df_2020_07)
df_merge23=merge_dataset(df_merge22,df_2020_08)
df_merge24=merge_dataset(df_merge23,df_2020_09)

## Convert str in df to list

In [321]:
def convert_to_str(df):
    purchasing_list=[]
    for i in range(len(df)):
        a=df['purchasing_list'][i].split(", ")
        purchasing_list.append(a)
    return purchasing_list

In [333]:
purchasing_freq_list=convert_to_str(df_merge24)

In [342]:
# unique purchasing record for each customer_id
purchasing_unique=remove_duplicate(purchasing_freq_list)

# most frequent item
most_frequent_item=most_frequent(purchasing_freq_list)

## Final output to csv

In [345]:
list_column=["customer_id","purchasing_list","most_frequent"]
column_A= df_merge24["customer_id"]
column_B= purchasing_unique
column_C=most_frequent_item
list_row=[]

    #to create a list full of rows as the writerow function reads data row-wise
for i in range(len(column_A)):
    list_temp=[column_A[i],column_B[i],column_C[i]]
    list_row.append(list_temp)


with open ('final_coutput.csv', 'w', newline="") as entry:
    writer=csv.writer(entry)
    writer.writerow(list_column)
    writer.writerows(list_row)

    entry.close()
        